In [1]:
import pandas as pd
import sqlite3

from scipy.stats import linregress
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from pykalman import KalmanFilter
import numpy as np

In [ ]:
#NOTE: bunch of memory duplication, but this is a notebook. don't convert to .py yet

In [2]:
bc_stations = pd.read_csv('./index_data/bc_hydro_stations.csv', sep=' \t', engine='python')
# we'll need some kind of entity resolution to cut down to reservoirs -> choose resolve stations to reservoirs (leg work)

In [ ]:
bc_stations

In [3]:
def filter_by_stations(df, stations):
    """
        Return only data with a station in stations
    """
    return df[df['STATION_NUMBER'].isin(stations)][:]

In [4]:
def plot_annual_averages(df, station_no):
    """
        lazy visual for annual - double check if we have weird outliers
    """
    station = df[df['STATION_NUMBER'] == station_no]
    fitline = linregress(station['YEAR'], station['MEAN'])
    # linregress val is pretty meaningless, but mostly there for context
    plt.plot(station['YEAR'], station['MEAN'])
    plt.plot(station['YEAR'], fitline.intercept + station['YEAR']*fitline.slope )
    
    plt.show()
#     return annual_linregressor

In [5]:
def plot_daily_values(df, kalman_smoothed):
    """
        visualize basic model regresssions
    """
    loess_smoothed = lowess(df['value'], df['date'], frac=0.03, is_sorted=False)
    # again, this is mostly visual
    plt.figure(figsize=(50, 20))
    plt.plot(df['date'], df['value'], 'b.', alpha=0.5)
    plt.plot(df['date'], loess_smoothed[:, 1], 'r-')
    plt.plot(df['date'], kalman_smoothed[:, 0], 'g-')
    plt.show()

In [6]:
def get_stations_daily_data(conn, stations):
    #only works for single station now, later groupby
    stations_str = str(stations)[1:-1]
    LEVELS = ['LEVEL{}'.format(dayno) for dayno in range(1,32)]
    query = "SELECT {levels} FROM DLY_LEVELS WHERE STATION_NUMBER IN ({stations})".format(levels=LEVELS, stations=stations_str)
    print(query)
    unparsed_data = pd.read_sql_query(daily_levels_query, conn)
    return unparsed_data

In [7]:
# see http://collaboration.cmc.ec.gc.ca/cmc/hydrometrics/www/HYDAT_Definition_EN.pdf for HYDAT schema
db_filename = './data/Hydat.sqlite3'
conn = sqlite3.connect(db_filename)

In [8]:
annual_query = "SELECT * FROM `ANNUAL_STATISTICS` WHERE `DATA_TYPE` = 'H';"
good_annual_stations_query = """SELECT STATION_NUMBER
                                FROM ANNUAL_STATISTICS 
                                WHERE DATA_TYPE = 'H'
                                    AND MEAN IS NOT NULL
                                GROUP BY STATION_NUMBER
                                HAVING COUNT(*) > 10;"""
bc_stations_str = str(list(bc_stations['Station Number']))[1:-1]
daily_levels_query = "SELECT * FROM DLY_LEVELS WHERE STATION_NUMBER IN ({stations})".format(stations=bc_stations_str)

In [9]:
annual_data = pd.read_sql_query(annual_query,conn) #anual statistics levels table
good_stations = pd.read_sql_query(good_annual_stations_query, conn) # stations with 'enough' data (>10 avg years)
daily_data = pd.read_sql_query(daily_levels_query, conn)#.set_index(['STATION_NUMBER','YEAR', 'MONTH']) # stations with 'enough' data (>10 avg years)

In [10]:
cleaned_annual_data = filter_by_stations(annual_data, good_stations['STATION_NUMBER']).dropna(subset=['MEAN'])
# stations w/enough years, remove years w/bananas

In [ ]:
# for station_no in good_stations['STATION_NUMBER']:
#     plot_annual_averages(cleaned_annual_data, station_no)

In [11]:
good_stations

,STATION_NUMBER
0,01AD004
1,01AK003
2,01AO002
3,01AO003
4,01AO004
5,01AO010
6,01AO011
7,01AP003
8,01AP005
9,01AQ007


In [ ]:
my_station = '08GA079' # COQUITLAM LAKE FOREBAY
plot_annual_averages(cleaned_annual_data, my_station)

In [ ]:
daily_data
# look at max/min later on for outlier-checking or to better inform
# Later, try Kalman filter w/weather... nulls in Kalman
# there is a fullmonth and no-days tag - maybe take advantage if gapped data is too hard
# ignoring symbols, check later to see if important (may account for inaccuracies)
# they gives us precision (in m, to cm or mm), ignoring for now

In [ ]:
station_day_data = filter_by_stations(daily_data, [my_station])#daily_data.loc[[my_station]]
melted_day_data = station_day_data.melt(id_vars = ['YEAR', 'MONTH'], value_vars=LEVELS).dropna()
melted_day_data['date'] = pd.to_datetime(melted_day_data['YEAR'].map(str) + '-' + melted_day_data['MONTH'].map(str)+ '-' + melted_day_data['variable'].str[5:])
melted_day_data.set_index('date', inplace=True)
melted_day_data.sort_index(inplace=True)
melted_day_data.reset_index(inplace=True)

In [ ]:
df = melted_day_data[:]
df['date'] = df['date'].astype(np.int64) // 1e9 # convert dates to time
df

In [ ]:
kalman_data = df[['value', 'date']]
initial_state = kalman_data.iloc[0]
observation_covariance = np.diag([1, 0.01]) ** 2
transition_covariance = np.diag([0.5, 0.02]) ** 2
transition = [[1, 0], [0, 1]]
# work out the proper values later. add weather to this later on - will inform precip

kf = KalmanFilter(initial_state_mean=initial_state,
                  observation_covariance=observation_covariance,
                  transition_covariance=transition_covariance,
                  transition_matrices=transition)
kalman_smoothed, _ = kf.smooth(kalman_data)

plot_daily_values(df, kalman_smoothed)


In [ ]:
# SELECT S.*, D.YEAR_FROM, D.YEAR_TO, D.RECORD_LENGTH FROM
# (SELECT STATION_NUMBER, STATION_NAME, LATITUDE, LONGITUDE, DRAINAGE_AREA_GROSS FROM STATIONS WHERE STATIONS.PROV_TERR_STATE_LOC='BC') S
# INNER JOIN
# (SELECT STATION_NUMBER, YEAR_FROM, YEAR_TO, RECORD_LENGTH FROM STN_DATA_RANGE WHERE DATA_TYPE = 'H' AND YEAR_FROM <= 1990 AND YEAR_TO >= 2015 AND RECORD_LENGTH>5) D
# ON S.STATION_NUMBER = D.STATION_NUMBER;
# -> need to look at data and filter 

# annual_query = "SELECT * FROM `ANNUAL_STATISTICS` WHERE `DATA_TYPE` = 'H';"
# good_annual_stations_query = """SELECT STATION_NUMBER
#                                 FROM ANNUAL_STATISTICS 
#                                 WHERE DATA_TYPE = 'H'
#                                     AND MEAN IS NOT NULL
#                                 GROUP BY STATION_NUMBER
#                                 HAVING COUNT(*) > 10;"""
# bc_stations_str = str(list(bc_stations['Station Number']))[1:-1]
# daily_levels_query = "SELECT * FROM DLY_LEVELS WHERE STATION_NUMBER IN ({stations})".format(stations=bc_stations_str)